<a href="https://colab.research.google.com/github/calebarr/AIS/blob/ais_cleaning/AIS_Downloader_TeamShared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AIS Data Downloader - Google Colab Version (Team-Ready)

# Mount Google Drive
from google.colab import drive
import os
from datetime import datetime, timedelta
import requests

print("Mounting Google Drive...")
drive.mount('/content/drive')

# Define the AIS downloader function
def download_ais_data(start_date_str, end_date_str, save_folder):
    os.makedirs(save_folder, exist_ok=True)
    print(f"Files will be saved to: {save_folder}")

    # Convert string dates to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    # Generate the date list
    dates_to_download = [
        (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
        for i in range((end_date - start_date).days + 1)
    ]

    # Download each file in the date range
    for date_str in dates_to_download:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        filename = f"AIS_{date_obj.strftime('%Y_%m_%d')}.zip"
        url = f"https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2020/{filename}"

        print(f"Downloading {filename}...")
        response = requests.get(url)

        if response.status_code == 200:
            file_path = os.path.join(save_folder, filename)
            with open(file_path, "wb") as f:
                f.write(response.content)
            print(f"Saved: {file_path}")
        else:
            print(f"Failed to download {filename} (HTTP {response.status_code})")

# Now call the function with your chosen parameters
download_ais_data(
    start_date_str="2020-01-05",
    end_date_str="2020-01-06",
    save_folder="/content/drive/My Drive/SIADS_593/assets/raw_data"
)


Mounting Google Drive...
Mounted at /content/drive
Files will be saved to: /content/drive/My Drive/SIADS_593/assets/raw_data
Saved: /content/drive/My Drive/SIADS_593/assets/raw_data/AIS_2020_01_05.zip
Saved: /content/drive/My Drive/SIADS_593/assets/raw_data/AIS_2020_01_06.zip


In [2]:
!ls -lh "/content/drive/My Drive/SIADS_593/assets/raw_data"


total 1.3G
-rw------- 1 root root 252M May 21 06:02 AIS_2020_01_01.zip
-rw------- 1 root root 250M May 21 06:02 AIS_2020_01_02.zip
-rw------- 1 root root 256M May 21 06:02 AIS_2020_01_03.zip
-rw------- 1 root root 245M May 27 06:55 AIS_2020_01_05.zip
-rw------- 1 root root 254M May 27 06:55 AIS_2020_01_06.zip


In [4]:
import zipfile
import pandas as pd

def inspect_zip_sample(zip_path, nrows=5):

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_files = [name for name in zip_ref.namelist() if name.endswith('.csv')]
        print("Files inside zip:", csv_files)

        with zip_ref.open(csv_files[0]) as f:
            df = pd.read_csv(f, nrows=nrows)

    print("Columns:", df.columns.tolist())
    return df



In [6]:
sample_df = inspect_zip_sample("/content/drive/My Drive/SIADS_593/assets/raw_data/AIS_2020_01_05.zip")
sample_df.head()

Files inside zip: ['AIS_2020_01_05.csv']
Columns: ['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading', 'VesselName', 'IMO', 'CallSign', 'VesselType', 'Status', 'Length', 'Width', 'Draft', 'Cargo', 'TransceiverClass']


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
0,367005220,2020-01-05T00:00:00,30.06265,-93.35127,5.1,227.0,511.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,A
1,215560000,2020-01-05T00:00:00,34.64002,-121.01854,16.2,334.2,342.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,A
2,636018760,2020-01-05T00:00:00,28.78695,-116.24974,12.8,343.4,345.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,A
3,368099320,2020-01-05T00:00:00,30.05953,-93.38022,4.4,270.4,511.0,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,A
4,368071490,2020-01-05T00:00:00,41.67222,-88.02777,0.0,300.1,247.0,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,A


In [10]:
import zipfile
import pandas as pd
import os

def load_csvs_from_zips(folder_path="/content/drive/My Drive/SIADS_593/assets/raw_data/"):

    all_dfs = []

    for file in os.listdir(folder_path):
        if file.endswith('.zip'):
            zip_path = os.path.join(folder_path, file)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                for csv_file in zip_ref.namelist():
                    if csv_file.endswith('.csv'):
                        with zip_ref.open(csv_file) as f:
                            df = pd.read_csv(f)
                            all_dfs.append(df)

    return pd.concat(all_dfs, ignore_index=True)


In [11]:
df = load_csvs_from_zips()  # uses the default folder



In [13]:
import zipfile
import pandas as pd

def process_zip_in_chunks(zip_path, chunksize=100_000):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_files = [name for name in zip_ref.namelist() if name.endswith('.csv')]
        if not csv_files:
            print(f"No CSV found in {zip_path}")
            return

        with zip_ref.open(csv_files[0]) as f:
            reader = pd.read_csv(f, chunksize=chunksize)

            for i, chunk in enumerate(reader):
                print(f"Processing chunk {i} with {len(chunk)} rows")
                # Example: just print basic info
                print(chunk.head(2))  # Replace with your real logic


In [14]:
process_zip_in_chunks("/content/drive/My Drive/SIADS_593/assets/raw_data/AIS_2020_01_05.zip")


Processing chunk 0 with 100000 rows
        MMSI         BaseDateTime       LAT        LON   SOG    COG  Heading  \
0  367005220  2020-01-05T00:00:00  30.06265  -93.35127   5.1  227.0    511.0   
1  215560000  2020-01-05T00:00:00  34.64002 -121.01854  16.2  334.2    342.0   

  VesselName  IMO CallSign  VesselType  Status  Length  Width  Draft  Cargo  \
0        NaN  NaN      NaN         NaN     0.0     NaN    NaN    NaN    NaN   
1        NaN  NaN      NaN         NaN     0.0     NaN    NaN    NaN    NaN   

  TransceiverClass  
0                A  
1                A  
Processing chunk 1 with 100000 rows
             MMSI         BaseDateTime       LAT        LON   SOG    COG  \
100000  372192000  2020-01-05T00:21:53  30.80373 -119.40367  10.8   70.0   
100001  366939780  2020-01-05T00:21:54  40.66074  -74.07110   0.0  245.6   

        Heading VesselName         IMO CallSign  VesselType  Status  Length  \
100000     69.0     KUMANO  IMO9333761    3EHO5        70.0     0.0   174.0   